# CliffWalking-v0

In [1]:
import sys
import logging
import imp
import itertools

import numpy as np
np.random.seed(0)
import scipy
import gym

imp.reload(logging)
logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

### Use Environment

import environment

In [2]:
env = gym.make('CliffWalking-v0')
env.seed(0)
logging.info('observation_space = %s', env.observation_space)
logging.info('action_space = %s', env.action_space)
logging.info('number of states = %s, number of actions = %s', env.nS, env.nA)
logging.info('shape of map = %s', env.shape)

22:15:58 [INFO] observation_space = Discrete(48)
22:15:58 [INFO] action_space = Discrete(4)
22:15:58 [INFO] number of states = 48, number of actions = 4
22:15:58 [INFO] shape of map = (4, 12)


Play an episode

In [3]:
def play_once(env, policy):
    total_reward = 0
    state = env.reset()
    while True:
        loc = np.unravel_index(state, env.shape)
        action = np.random.choice(env.nA, p=policy[state])
        next_state, reward, done, _ = env.step(action)
        logging.debug(
                'state = %s, location = %s, action = %s, reward = %s',
                state, loc, action, reward)
        total_reward += reward
        if done:
            break
        state = next_state
    return total_reward

Play an episode using optimal policy

In [4]:
actions = np.ones(env.shape, dtype=int)
actions[-1, :] = 0
actions[:, -1] = 2
optimal_policy = np.eye(4)[actions.reshape(-1)]

In [5]:
episode_reward = play_once(env, optimal_policy)
logging.info('episode_reward = %s', episode_reward)

22:15:58 [DEBUG] state = 36, location = (3, 0), action = 0, reward = -1
22:15:58 [DEBUG] state = 24, location = (2, 0), action = 1, reward = -1
22:15:58 [DEBUG] state = 25, location = (2, 1), action = 1, reward = -1
22:15:58 [DEBUG] state = 26, location = (2, 2), action = 1, reward = -1
22:15:58 [DEBUG] state = 27, location = (2, 3), action = 1, reward = -1
22:15:58 [DEBUG] state = 28, location = (2, 4), action = 1, reward = -1
22:15:58 [DEBUG] state = 29, location = (2, 5), action = 1, reward = -1
22:15:58 [DEBUG] state = 30, location = (2, 6), action = 1, reward = -1
22:15:58 [DEBUG] state = 31, location = (2, 7), action = 1, reward = -1
22:15:58 [DEBUG] state = 32, location = (2, 8), action = 1, reward = -1
22:15:58 [DEBUG] state = 33, location = (2, 9), action = 1, reward = -1
22:15:58 [DEBUG] state = 34, location = (2, 10), action = 1, reward = -1
22:15:58 [DEBUG] state = 35, location = (2, 11), action = 2, reward = -1
22:15:58 [INFO] episode_reward = -13


### Solve Bellman Expectation Function

In [6]:
def evaluate_bellman(env, policy, gamma=1.):
    a, b = np.eye(env.nS), np.zeros((env.nS))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            pi = policy[state][action]
            for p, next_state, reward, done in env.P[state][action]:
                a[state, next_state] -= (pi * gamma * p)
                b[state] += (pi * reward * p)
    v = np.linalg.solve(a, b)
    q = np.zeros((env.nS, env.nA))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            for p, next_state, reward, done in env.P[state][action]:
                q[state][action] += ((reward + gamma * v[next_state]) * p)
    return v, q

Evaluate Random Policy

In [7]:
policy = np.random.uniform(size=(env.nS, env.nA))
policy = policy / np.sum(policy, axis=1)[:, np.newaxis]

state_values, action_values = evaluate_bellman(env, policy)
logging.info('state value = %s', state_values)
logging.info('action value = %s', action_values)

22:15:58 [INFO] state value = [-71259.73993047 -71237.78971177 -71194.84815308 -71140.80283085
 -71061.70657408 -70938.91093031 -68108.48102394 -66319.1066515
 -64517.35282333 -61466.53107192 -57841.3923356  -49099.2438373
 -71264.9652292  -71247.53092066 -71217.52824197 -71120.56871129
 -71054.59876057 -70900.38265544 -69089.02487541 -67348.61705427
 -65077.28575315 -59299.02626943 -54477.04027156 -43408.92846114
 -71343.97588505 -71365.86170719 -71356.29488212 -71310.5189658
 -71248.19189086 -70916.43626143 -70432.29357882 -69519.49523581
 -69113.60088441 -66843.01499869 -60740.99702848 -33361.65231115
 -71417.87145991 -71463.16859934 -71447.66816434 -71480.47316828
 -71514.26925749 -71213.20010574 -71233.56329317 -71021.31099415
 -70840.1582772  -69680.13842415 -65008.34042541      0.        ]
22:15:58 [INFO] action value = [[-7.12607399e+04 -7.12387897e+04 -7.12659652e+04 -7.12607399e+04]
 [-7.12387897e+04 -7.11958482e+04 -7.12485309e+04 -7.12607399e+04]
 [-7.11958482e+04 -7.114180

Evaluate Optimal Policy

In [8]:
optimal_state_values, optimal_action_values = evaluate_bellman(env, optimal_policy)
logging.info('optimal state value = %s', optimal_state_values)
logging.info('optimal action value = %s', optimal_action_values)

22:15:58 [INFO] optimal state value = [-14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3. -13. -12.
 -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2. -12. -11. -10.  -9.
  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1. -13. -12. -11. -10.  -9.  -8.
  -7.  -6.  -5.  -4.  -3.   0.]
22:15:58 [INFO] optimal action value = [[ -15.  -14.  -14.  -15.]
 [ -14.  -13.  -13.  -15.]
 [ -13.  -12.  -12.  -14.]
 [ -12.  -11.  -11.  -13.]
 [ -11.  -10.  -10.  -12.]
 [ -10.   -9.   -9.  -11.]
 [  -9.   -8.   -8.  -10.]
 [  -8.   -7.   -7.   -9.]
 [  -7.   -6.   -6.   -8.]
 [  -6.   -5.   -5.   -7.]
 [  -5.   -4.   -4.   -6.]
 [  -4.   -4.   -3.   -5.]
 [ -15.  -13.  -13.  -14.]
 [ -14.  -12.  -12.  -14.]
 [ -13.  -11.  -11.  -13.]
 [ -12.  -10.  -10.  -12.]
 [ -11.   -9.   -9.  -11.]
 [ -10.   -8.   -8.  -10.]
 [  -9.   -7.   -7.   -9.]
 [  -8.   -6.   -6.   -8.]
 [  -7.   -5.   -5.   -7.]
 [  -6.   -4.   -4.   -6.]
 [  -5.   -3.   -3.   -5.]
 [  -4.   -3.   -2.   -4.]
 [ -14.  -12.  -14.  -13.]
 [ 

### Solve Bellman Optimal Equation

In [9]:
def optimal_bellman(env, gamma=1.):
    p = np.zeros((env.nS, env.nA, env.nS))
    r = np.zeros((env.nS, env.nA))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[state][action]:
                p[state, action, next_state] += prob
                r[state, action] += (reward * prob)
    c = np.ones(env.nS)
    a_ub = gamma * p.reshape(-1, env.nS) - \
            np.repeat(np.eye(env.nS), env.nA, axis=0)
    b_ub = -r.reshape(-1)
    a_eq = np.zeros((0, env.nS))
    b_eq = np.zeros(0)
    bounds = [(None, None),] * env.nS
    res = scipy.optimize.linprog(c, a_ub, b_ub, bounds=bounds,
            method='interior-point')
    v = res.x
    q = r + gamma * np.dot(p, v)
    return v, q

In [10]:
optimal_state_values, optimal_action_values = optimal_bellman(env)
logging.info('optimal state value = %s', optimal_state_values)
logging.info('optimal action value = %s', optimal_action_values)

22:15:58 [INFO] optimal state value = [-1.40000000e+01 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01
 -1.00000000e+01 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00
 -6.00000000e+00 -5.00000000e+00 -4.00000000e+00 -3.00000000e+00
 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01
 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00
 -5.00000000e+00 -4.00000000e+00 -3.00000000e+00 -2.00000000e+00
 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01 -9.00000000e+00
 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00 -5.00000000e+00
 -4.00000000e+00 -3.00000000e+00 -2.00000000e+00 -1.00000000e+00
 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01
 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00
 -5.00000000e+00 -4.00000000e+00 -9.99999999e-01  1.82540720e-11]
22:15:58 [INFO] optimal action value = [[ -14.99999999  -13.99999999  -13.99999999  -14.99999999]
 [ -13.99999999  -13.          -13.          -14.99999999]
 [ -13.          -12.   

In [11]:
optimal_actions = optimal_action_values.argmax(axis=1)
logging.info('optimal policy = %s', optimal_actions)

22:15:58 [INFO] optimal policy = [2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 0
 0 0 0 0 0 0 0 0 0 1 0]
